In [ ]:
from elasticsearch import Elasticsearch, helpers
import utils.util as util

es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])
merged_records = merged_df.to_dict(orient='records')
helpers.bulk(es, util.index_data(merged_records, index='cluster_ratings'))

index_name = "cluster_ratings"

# The Elasticsearch query
response = es.search(index=index_name, body={
    "size": 0,
    "aggs": {
        "by_cluster": {
            "terms": {
                "field": "cluster",
                "size": 200
            },
            "aggs": {
                "by_isbn": {
                    "terms": {
                        "field": "isbn",
                        "size": 10000
                    },
                    "aggs": {
                        "avg_rating": {
                            "avg": {
                                "field": "rating"
                            }
                        }
                    }
                }
            }
        }
    }
})

# Parse results
clusters = response['aggregations']['by_cluster']['buckets']

# To extract meaningful information:
for cluster in clusters:
    cluster_id = cluster['key']
    books = cluster['by_isbn']['buckets']
    for book in books:
        isbn = book['key']
        avg_rating = book['avg_rating']['value']
        print(f"Cluster: {cluster_id}, ISBN: {isbn}, Avg Rating: {avg_rating}")